In [0]:
from pyspark.sql.functions import col, when, trim, lower, coalesce, count
from pyspark.sql.types import IntegerType, FloatType, StringType

In [0]:
# Load raw data
df_raw = spark.read.table("samsung_raw")

print("=" * 80)
print("SAMSUNG MOBILES - RAW DATA ANALYSIS")
print("=" * 80)

# Display schema and basic info
display(df_raw.limit(10))
print(f"Total rows: {df_raw.count()}")
print(f"Total columns: {len(df_raw.columns)}")

print("\nColumn Names and Types:")
df_raw.printSchema()

# Check for duplicates
print(f"\nDuplicate records: {df_raw.count() - df_raw.dropDuplicates().count()}")

# Check for nulls in each column
print("\nNull values per column:")
null_counts = df_raw.select([count(when(col(c).isNull(), c)).alias(c) for c in df_raw.columns])
display(null_counts)
""


SAMSUNG MOBILES - RAW DATA ANALYSIS


name,ratings,price,imgURL,camera,display,battery,storage,ram,processor,android_version
"SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)",4.3,9499,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/x/x/s/-original-imagfhu6dcpdnqkh.jpeg?q=70,50,other_types,6000,64,4,not mentioned,12
"SAMSUNG Galaxy F13 (Waterfall Blue, 128 GB)",4.3,10499,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/0/8/4/-original-imagfhu75eupxyft.jpeg?q=70,50,other_types,6000,128,4,not mentioned,12
"SAMSUNG Galaxy F13 (Nightsky Green, 128 GB)",4.3,10499,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/a/i/v/-original-imagfhu6bdzhnmkz.jpeg?q=70,50,other_types,6000,128,4,not mentioned,12
"SAMSUNG Galaxy F13 (Sunrise Copper, 128 GB)",4.3,10499,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/x/x/s/-original-imagfhu6dcpdnqkh.jpeg?q=70,50,other_types,6000,128,4,not mentioned,12
"SAMSUNG Galaxy F23 5G (Aqua Blue, 128 GB)",4.3,15499,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/0/h/k/-original-imagcg22czc3ggvw.jpeg?q=70,50,other_types,5000,128,6,Snapdragon750GProcessor,12
"SAMSUNG Galaxy F23 5G (Forest Green, 128 GB)",4.3,15499,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/o/p/z/-original-imagcg22pf79cgau.jpeg?q=70,50,other_types,5000,128,6,Snapdragon750GProcessor,12
"SAMSUNG Galaxy F04 (Jade Purple, 64 GB)",4.2,6999,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/7/p/9/-original-imaghgbyypzkzvud.jpeg?q=70,13,LCD,5000,64,4,not mentioned,12
"SAMSUNG Galaxy F04 (Opal Green, 64 GB)",4.2,6999,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/w/d/o/-original-imaghgbyhy6banxv.jpeg?q=70,13,LCD,5000,64,4,not mentioned,12
"SAMSUNG Galaxy F14 5G (GOAT Green, 128 GB)",4.2,13990,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/j/i/k/galaxy-f14-5g-sm-e146bzggins-samsung-original-imagnzdkprfwj9hv.jpeg?q=70,50,LCD,6000,128,6,not mentioned,13
"SAMSUNG Galaxy F23 5G (Copper Blush, 128 GB)",4.3,15499,https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/f/c/v/-original-imagehzbaw2wugme.jpeg?q=70,50,other_types,5000,128,6,Snapdragon750GProcessor,12


Total rows: 407
Total columns: 11

Column Names and Types:
root
 |-- name: string (nullable = true)
 |-- ratings: double (nullable = true)
 |-- price: long (nullable = true)
 |-- imgURL: string (nullable = true)
 |-- camera: long (nullable = true)
 |-- display: string (nullable = true)
 |-- battery: long (nullable = true)
 |-- storage: long (nullable = true)
 |-- ram: long (nullable = true)
 |-- processor: string (nullable = true)
 |-- android_version: long (nullable = true)


Duplicate records: 0

Null values per column:


name,ratings,price,imgURL,camera,display,battery,storage,ram,processor,android_version
0,0,0,0,0,0,0,0,0,0,0


''

In [0]:

from pyspark.sql.functions import col, when, trim, lower, round, desc, substring

print("\n" + "=" * 80)
print("DATA CLEANING & TRANSFORMATION")
print("=" * 80)

# Step 1: Remove duplicates
df_cleaned = df_raw.dropDuplicates()
print(f"✓ Removed duplicates. Records: {df_cleaned.count()}")

# Step 2: Fix data types
df_cleaned = df_cleaned.withColumn("price", col("price").cast("double")) \
                        .withColumn("ratings", col("ratings").cast("double")) \
                        .withColumn("camera", col("camera").cast("int")) \
                        .withColumn("battery", col("battery").cast("int")) \
                        .withColumn("storage", col("storage").cast("int")) \
                        .withColumn("ram", col("ram").cast("int")) \
                        .withColumn("android_version", col("android_version").cast("int"))

print("✓ Data types corrected")



DATA CLEANING & TRANSFORMATION
✓ Removed duplicates. Records: 407
✓ Data types corrected


In [0]:
original_count = df_cleaned.count()
df_cleaned = df_cleaned.filter((col("price") > 0) & (col("ratings") > 0) & (col("ratings") <= 5))
print(f"✓ Removed invalid rows. Filtered: {original_count - df_cleaned.count()}")

# Step 4: Standardize text columns
df_cleaned = df_cleaned.withColumn("name", trim(lower(col("name")))) \
                        .withColumn("processor", trim(lower(col("processor")))) \
                        .withColumn("display", trim(lower(col("display"))))

print("✓ Standardized text columns")

# Step 5: Create derived columns for business logic
df_cleaned = df_cleaned.withColumn(
    "price_category",
    when(col("price") < 15000, "Budget")
    .when(col("price") < 30000, "Mid-Range")
    .when(col("price") < 50000, "Premium")
    .otherwise("Ultra-Premium")
)

df_cleaned = df_cleaned.withColumn(
    "rating_category",
    when(col("ratings") < 3.0, "Low")
    .when(col("ratings") < 4.0, "Medium")
    .otherwise("High")
)

# Extract series from product name
df_cleaned = df_cleaned.withColumn(
    "series",
    when(col("name").contains("galaxy a"), "Galaxy A")
    .when(col("name").contains("galaxy s"), "Galaxy S")
    .when(col("name").contains("galaxy z"), "Galaxy Z")
    .when(col("name").contains("galaxy m"), "Galaxy M")
    .when(col("name").contains("galaxy f"), "Galaxy F")
    .otherwise("Other")
)

# Create specs score (composite metric)
df_cleaned = df_cleaned.withColumn(
    "specs_score",
    round((
        (col("camera") / 50) +
        (col("ram") / 12) +
        (col("storage") / 512) +
        (col("battery") / 6000)
    ) / 4 * 100, 2)
)

# Create value ratio
df_cleaned = df_cleaned.withColumn(
    "value_ratio",
    round(col("ratings") / (col("price") / 10000), 4)
)

print("✓ Created derived columns:")
print("  - price_category")
print("  - rating_category")
print("  - series")
print("  - specs_score")
print("  - value_ratio")

# Display cleaned data sample
print("\nCleaned Data Sample:")
display(df_cleaned.select("name", "series", "price", "price_category", "ratings", "rating_category", "specs_score").limit(10))

print(f"\n✓ Final record count: {df_cleaned.count()}")
print("✓ DATA CLEANING COMPLETE!")
""


✓ Removed invalid rows. Filtered: 0
✓ Standardized text columns
✓ Created derived columns:
  - price_category
  - rating_category
  - series
  - specs_score
  - value_ratio

Cleaned Data Sample:


name,series,price,price_category,ratings,rating_category,specs_score
"samsung galaxy f13 (sunrise copper, 64 gb)",Galaxy F,9499.0,Budget,4.3,High,61.46
"samsung galaxy f13 (waterfall blue, 128 gb)",Galaxy F,10499.0,Budget,4.3,High,64.58
"samsung galaxy f13 (nightsky green, 128 gb)",Galaxy F,10499.0,Budget,4.3,High,64.58
"samsung galaxy f13 (sunrise copper, 128 gb)",Galaxy F,10499.0,Budget,4.3,High,64.58
"samsung galaxy f23 5g (aqua blue, 128 gb)",Galaxy F,15499.0,Mid-Range,4.3,High,64.58
"samsung galaxy f23 5g (forest green, 128 gb)",Galaxy F,15499.0,Mid-Range,4.3,High,64.58
"samsung galaxy f04 (jade purple, 64 gb)",Galaxy F,6999.0,Budget,4.2,High,38.79
"samsung galaxy f04 (opal green, 64 gb)",Galaxy F,6999.0,Budget,4.2,High,38.79
"samsung galaxy f14 5g (goat green, 128 gb)",Galaxy F,13990.0,Budget,4.2,High,68.75
"samsung galaxy f23 5g (copper blush, 128 gb)",Galaxy F,15499.0,Mid-Range,4.3,High,64.58



✓ Final record count: 407
✓ DATA CLEANING COMPLETE!


''

In [0]:
from pyspark.sql.functions import desc

print("\n" + "=" * 80)
print("SAVING DATA TO DELTA TABLES (BRONZE-SILVER-GOLD ARCHITECTURE)")
print("=" * 80)

# SILVER Layer: Cleaned data
df_cleaned.write.format("delta").mode("overwrite") \
    .saveAsTable("samsung_silver")
print("✓ Created SILVER layer: samsung_silver")

# GOLD Layer 1: Performance metrics
performance_metrics = df_cleaned.select(
    "name", "series", "price", "price_category", "ratings", "rating_category",
    "camera", "ram", "storage", "battery", "specs_score", "value_ratio"
).orderBy(desc("ratings"))

performance_metrics.write.format("delta").mode("overwrite") \
    .saveAsTable("performance_metrics")
print("✓ Created GOLD layer 1: performance_metrics")

# GOLD Layer 2: Price analysis
price_analysis = spark.sql("""
SELECT 
    price_category,
    series,
    COUNT(*) as model_count,
    ROUND(AVG(price), 2) as avg_price,
    ROUND(MIN(price), 2) as min_price,
    ROUND(MAX(price), 2) as max_price,
    ROUND(AVG(ratings), 2) as avg_rating,
    ROUND(AVG(ram), 2) as avg_ram,
    ROUND(AVG(battery), 0) as avg_battery
FROM samsung_silver
WHERE series != 'Other'
GROUP BY price_category, series
ORDER BY price_category, avg_rating DESC
""")

price_analysis.write.format("delta").mode("overwrite") \
    .saveAsTable("price_analysis")
print("✓ Created GOLD layer 2: price_analysis")

# GOLD Layer 3: Series benchmark
series_benchmark = spark.sql("""
SELECT 
    series,
    COUNT(*) as total_models,
    ROUND(AVG(price), 0) as avg_price,
    ROUND(MIN(price), 0) as min_price,
    ROUND(MAX(price), 0) as max_price,
    ROUND(AVG(ratings), 2) as avg_rating,
    ROUND(AVG(specs_score), 2) as avg_specs_score,
    COUNT(CASE WHEN ratings >= 4.3 THEN 1 END) as high_quality_models
FROM samsung_silver
GROUP BY series
ORDER BY total_models DESC
""")

series_benchmark.write.format("delta").mode("overwrite") \
    .saveAsTable("series_benchmark")
print("✓ Created GOLD layer 3: series_benchmark")

print("\n✓ ALL DELTA TABLES CREATED SUCCESSFULLY!")
print("\nData Architecture:")
print("  BRONZE: samsung_raw (raw data)")
print("  SILVER: samsung_silver (cleaned data)")
print("  GOLD:   performance_metrics, price_analysis, series_benchmark")


SAVING DATA TO DELTA TABLES (BRONZE-SILVER-GOLD ARCHITECTURE)
✓ Created SILVER layer: samsung_silver
✓ Created GOLD layer 1: performance_metrics
✓ Created GOLD layer 2: price_analysis
✓ Created GOLD layer 3: series_benchmark

✓ ALL DELTA TABLES CREATED SUCCESSFULLY!

Data Architecture:
  BRONZE: samsung_raw (raw data)
  SILVER: samsung_silver (cleaned data)
  GOLD:   performance_metrics, price_analysis, series_benchmark
